In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import random



In [ ]:
#Reading jokes csv
joke_texts=pd.read_csv('JokeText.csv')
joke_texts.head()


,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [ ]:
#Reading rating csv
user_ratings=pd.read_csv('UserRatings1.csv')
user_ratings.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


In [ ]:
def user_vector():
  length=len(user_ratings['JokeId']) #Number of jokes in dataset
  vector=[0]*length               #Creating user vector filled with 0

  #----------Here we can change number of jokes to rate----------
  number_of_jokes_to_rate=10
  #----------Here we can change number of jokes to rate----------

  jokes_indexes=random.sample(range(0, length), number_of_jokes_to_rate)  #Geting 10 random jokes to rate

  for i in range(number_of_jokes_to_rate):
    print('You are rating joke nr', {i+1})
    print('Rate this joke on a scale -10 to 10:' )
    print()
    print(joke_texts.iloc[jokes_indexes[i]]['JokeText']) #Writing a joke from joke indexes
    while True:
      try:
        rate=input('Rating: ')
        rate=float(rate)
        rate_rounded=round(rate, 2)

        if -10<=rate_rounded<=10: #Condition for accepting given rating
          print("Saved rating: ", {rate_rounded})
          vector[jokes_indexes[i]]=rate_rounded #Updating new user rating vector
          break
        else:
          print("Error: The number should be from range -10 to 10. Try again")

      except ValueError: #If someone inputs not a number
        print("This is not a number. Try again")

  return vector

In [ ]:
from sklearn.neighbors import NearestNeighbors
def recomend_similar_users(user_vector):
  user_ratings.fillna(value=0, inplace=True)
  user_ratings['new_user']=user_vector
  #Users as rows without JokeId
  data_only=user_ratings.drop('JokeId',axis=1).T

  model_ratings=NearestNeighbors(metric='cosine', algorithm='brute')
  model_ratings.fit(data_only)

  distances, indices = model_ratings.kneighbors(data_only.iloc[-1, :].values.reshape(1, -1), n_neighbors=5)
  #Geting indices of n_neighbors without user_vector
  similar_users_indices = indices.flatten()[1:]

  return similar_users_indices


In [ ]:
def recomend_similar_joke(similar_users_indices):
  index=similar_users_indices[0] + 1  #+1 because now we take in account JokeID row
  similar_user_vector=user_ratings.iloc[:,index]

  jokes_not_rated_by_user=(user_ratings['new_user']==0) #Here we have jokes with rating 0 (it means that it was not rated by user)

  joke_id_recomendation=similar_user_vector[jokes_not_rated_by_user].idxmax()

  recomended_joke_text=joke_texts.loc[joke_id_recomendation, 'JokeText']
  print()
  print("----------------Based on your previous ratings the next joke is----------------")
  print()
  print()

  return recomended_joke_text


# Source information and legal disclaimer
print("-" * 60)
print("Data Source: These jokes were retrieved from Kaggle.")
print("Link: https://www.kaggle.com/datasets/aakaashjois/jester-collaborative-filtering-dataset")
print("\n[NOTE]: The jokes presented are for humorous purposes only.")
print("The content is not intended to offend or discriminate against anyone.")
print("-" * 60)



print(recomend_similar_joke(recomend_similar_users(user_vector())))

You are rating joke nr {1}
Rate this joke on a scale -10 to 10:

Reaching the end of a job interview, the human resources person asked a
young engineer fresh out of Stanford,

"And what starting salary were you looking for?"

The engineer said, "In the neighborhood of $125,000 a year, depending
on the benefits package."

The interviewer said, "Well, what would you say to a package of 5-weeks 
vacation, 14 paid holidays, full medical and dental, company matching 
retirement fund to 50% of salary, and a company car leased every 2 years - 
say, a red Corvette?"

The Engineer sat up straight and said, "Wow! Are you kidding?"

And the interviewer replied, "Yeah, but you started it."

Rating: 6
Saved rating:  {6.0}
You are rating joke nr {2}
Rate this joke on a scale -10 to 10:

Early one morning a mother went to her sleeping son and woke him up.

"Wake up, son.  It's time to go to school." 
"But why, Mama?  I don't want to go to school." 
"Give me two reasons why you don't want to go to sch